In [1]:
%pwd

'C:\\NN-P4\\python-notebooks'

###Imports and Read CSV

In [2]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime
from keras.callbacks import History # to get train accuracy
history = History()

In [3]:
#nPrintML creates a copy of dataset in pkl picles format. Convert to CSV 1st.
# Read the PKL
# df_train = pd.read_pickle('../datasets/nprint-raw/country-of-origin/X.pkl')
# df_test = pd.read_pickle('../datasets/nprint-raw/country-of-origin/X_val.pkl')
# label_train = pd.read_pickle('../datasets/nprint-raw/country-of-origin/y.pkl')
# label_test = pd.read_pickle('../datasets/nprint-raw/country-of-origin/y_val.pkl')
# # Save a copy as CSV
# df_train.to_csv('../datasets/nprint-raw/country-of-origin/X.csv')
# df_test.to_csv('../datasets/nprint-raw/country-of-origin/X_val.csv')
# label_train.to_csv('../datasets/nprint-raw/country-of-origin/y.csv')
# label_test.to_csv('../datasets/nprint-raw/country-of-origin/y_val.csv')
# Read CSV
df_train = pd.read_csv('../datasets/nprint-raw/country-of-origin/X.csv')
df_test = pd.read_csv('../datasets/nprint-raw/country-of-origin/X_val.csv')
label_train = pd.read_csv('../datasets/nprint-raw/country-of-origin/y.csv')
label_test = pd.read_csv('../datasets/nprint-raw/country-of-origin/y_val.csv')
#Drop colum with pcap sequence number
df_train.drop('pcap', axis=1, inplace=True)
df_test.drop('pcap', axis=1, inplace=True)
label_train.drop('pcap', axis=1, inplace=True)
label_test.drop('pcap', axis=1, inplace=True)
#Rename Unnamed Collumn
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)
#Read file with ordered list of attributes ranked by the autoML feature importance algorithm
features_rankings_df = pd.read_csv('../datasets/nprint-raw/country-of-origin/feature-importance.csv')
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

In [4]:
display(df_train)
display(df_test)
display(label_train)
display(label_test)
print(label_train["label"].value_counts())
print(label_test["label"].value_counts())
print(f"df_train shape: {df_train.shape} \n columns: {df_train.columns.values}")
print(f"df_test shape: {df_test.shape} \n columns: {df_test.columns.values}")
display(features_rankings_df.head(40))

,pkt_0_ipv4_tos_6,pkt_0_ipv4_tl_6,pkt_0_ipv4_tl_7,pkt_0_ipv4_tl_8,pkt_0_ipv4_tl_9,pkt_0_ipv4_tl_10,pkt_0_ipv4_tl_11,pkt_0_ipv4_tl_12,pkt_0_ipv4_tl_13,pkt_0_ipv4_tl_14,...,pkt_9_udp_cksum_6,pkt_9_udp_cksum_7,pkt_9_udp_cksum_8,pkt_9_udp_cksum_9,pkt_9_udp_cksum_10,pkt_9_udp_cksum_11,pkt_9_udp_cksum_12,pkt_9_udp_cksum_13,pkt_9_udp_cksum_14,pkt_9_udp_cksum_15
0,0,0,0,0,0,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,0,0,0,1,0,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0,0,0,0,1,0,0,0,0,0,...,1,1,1,1,1,0,0,1,1,1
3,0,0,0,0,1,0,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,0,0,0,0,1,0,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,0,0,0,0,1,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
233,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
234,0,0,0,0,0,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
235,0,0,0,0,1,0,0,0,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


,pkt_0_ipv4_tos_6,pkt_0_ipv4_tl_6,pkt_0_ipv4_tl_7,pkt_0_ipv4_tl_8,pkt_0_ipv4_tl_9,pkt_0_ipv4_tl_10,pkt_0_ipv4_tl_11,pkt_0_ipv4_tl_12,pkt_0_ipv4_tl_13,pkt_0_ipv4_tl_14,...,pkt_9_udp_cksum_6,pkt_9_udp_cksum_7,pkt_9_udp_cksum_8,pkt_9_udp_cksum_9,pkt_9_udp_cksum_10,pkt_9_udp_cksum_11,pkt_9_udp_cksum_12,pkt_9_udp_cksum_13,pkt_9_udp_cksum_14,pkt_9_udp_cksum_15
0,0,0,0,0,1,0,0,0,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,1,1,1,1,1
2,0,0,0,0,0,1,1,0,1,1,...,0,0,1,1,1,1,1,1,1,0
3,0,0,0,0,0,1,1,1,1,0,...,1,1,0,0,1,0,1,0,1,1
4,0,0,0,0,1,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,0,0,0,0,1,0,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,0,0,0,0,1,1,1,0,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,0,0,0,0,0,1,1,0,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,0,0,0,0,1,0,0,1,1,0,...,0,1,1,0,0,1,1,0,1,1
9,0,0,0,0,1,1,1,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


,label
0,0
1,0
2,1
3,0
4,1
...,...
232,1
233,2
234,1
235,2


,label
0,2
1,1
2,2
3,2
4,2
5,2
6,2
7,2
8,1
9,1


label
1    97
0    70
2    70
Name: count, dtype: int64
label
1    24
2    18
0    18
Name: count, dtype: int64
df_train shape: (237, 4825) 
 columns: ['pkt_0_ipv4_tos_6' 'pkt_0_ipv4_tl_6' 'pkt_0_ipv4_tl_7' ...
 'pkt_9_udp_cksum_13' 'pkt_9_udp_cksum_14' 'pkt_9_udp_cksum_15']
df_test shape: (60, 4825) 
 columns: ['pkt_0_ipv4_tos_6' 'pkt_0_ipv4_tl_6' 'pkt_0_ipv4_tl_7' ...
 'pkt_9_udp_cksum_13' 'pkt_9_udp_cksum_14' 'pkt_9_udp_cksum_15']


,Unnamed: 0,importance,stddev,p_value,n,p99_high,p99_low
0,pkt_5_tcp_opt_64,0.100000,0.017568,1.147362e-08,10,0.118055,0.081945
1,pkt_9_tcp_opt_64,0.075000,0.014164,2.161733e-08,10,0.089556,0.060444
2,pkt_5_tcp_opt_65,0.066667,0.017568,3.849943e-07,10,0.084721,0.048612
3,pkt_8_tcp_opt_64,0.058333,0.011785,3.894775e-08,10,0.070445,0.046222
4,pkt_7_tcp_opt_64,0.058333,0.014164,1.911429e-07,10,0.072889,0.043777
5,pkt_7_tcp_opt_65,0.055000,0.017656,2.027185e-06,10,0.073145,0.036855
6,pkt_3_tcp_opt_64,0.053333,0.013147,2.175770e-07,10,0.066844,0.039822
7,pkt_3_ipv4_ttl_6,0.048333,0.018342,7.977694e-06,10,0.067183,0.029484
8,pkt_6_tcp_opt_64,0.046667,0.010541,1.025415e-07,10,0.057499,0.035834
9,pkt_8_tcp_sprt_14,0.041667,0.018002,2.235802e-05,10,0.060167,0.023166


In [5]:
#make a list with only the best features

feature_list = features_rankings_df['Unnamed: 0'].tolist()
feature_list = feature_list[:32]
print(feature_list)

#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
df_train = df_train[feature_list]
df_test = df_test[feature_list]

display(df_train)
display(df_test)

['pkt_5_tcp_opt_64', 'pkt_9_tcp_opt_64', 'pkt_5_tcp_opt_65', 'pkt_8_tcp_opt_64', 'pkt_7_tcp_opt_64', 'pkt_7_tcp_opt_65', 'pkt_3_tcp_opt_64', 'pkt_3_ipv4_ttl_6', 'pkt_6_tcp_opt_64', 'pkt_8_tcp_sprt_14', 'pkt_9_tcp_opt_95', 'pkt_8_ipv4_cksum_1', 'pkt_4_tcp_opt_64', 'pkt_6_tcp_opt_65', 'pkt_8_tcp_opt_65', 'pkt_0_ipv4_tl_11', 'pkt_9_tcp_dprt_15', 'pkt_0_ipv4_ttl_7', 'pkt_5_udp_dport_10', 'pkt_8_tcp_ackf_0', 'pkt_1_ipv4_tl_12', 'pkt_9_tcp_opt_33', 'pkt_8_tcp_wsize_12', 'pkt_6_ipv4_cksum_4', 'pkt_5_ipv4_cksum_6', 'pkt_9_tcp_dprt_10', 'pkt_1_udp_sport_13', 'pkt_8_tcp_seq_16', 'pkt_2_ipv4_id_13', 'pkt_1_udp_cksum_6', 'pkt_0_ipv4_tl_12', 'pkt_6_udp_cksum_9']


,pkt_5_tcp_opt_64,pkt_9_tcp_opt_64,pkt_5_tcp_opt_65,pkt_8_tcp_opt_64,pkt_7_tcp_opt_64,pkt_7_tcp_opt_65,pkt_3_tcp_opt_64,pkt_3_ipv4_ttl_6,pkt_6_tcp_opt_64,pkt_8_tcp_sprt_14,...,pkt_8_tcp_wsize_12,pkt_6_ipv4_cksum_4,pkt_5_ipv4_cksum_6,pkt_9_tcp_dprt_10,pkt_1_udp_sport_13,pkt_8_tcp_seq_16,pkt_2_ipv4_id_13,pkt_1_udp_cksum_6,pkt_0_ipv4_tl_12,pkt_6_udp_cksum_9
0,1,0,0,0,0,0,0,1,1,1,...,1,0,1,0,0,1,0,1,1,-1
1,0,1,0,-1,-1,-1,-1,1,1,-1,...,-1,1,0,1,-1,-1,1,-1,0,-1
2,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,1,0,-1,0,-1,1,0,0,1
3,1,0,0,1,1,0,-1,1,-1,0,...,1,0,1,0,1,0,0,1,0,0
4,0,0,0,0,0,0,-1,0,0,1,...,1,0,0,1,-1,1,0,-1,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,0,0,0,0,0,0,0,0,0,1,...,1,1,0,0,1,1,1,1,0,-1
233,-1,-1,-1,-1,1,1,-1,1,0,-1,...,-1,0,1,-1,0,-1,0,1,0,-1
234,0,0,1,0,-1,-1,0,1,0,0,...,1,0,0,1,1,0,1,1,1,-1
235,-1,1,-1,1,0,0,0,1,1,1,...,1,1,0,1,1,1,0,0,0,-1


,pkt_5_tcp_opt_64,pkt_9_tcp_opt_64,pkt_5_tcp_opt_65,pkt_8_tcp_opt_64,pkt_7_tcp_opt_64,pkt_7_tcp_opt_65,pkt_3_tcp_opt_64,pkt_3_ipv4_ttl_6,pkt_6_tcp_opt_64,pkt_8_tcp_sprt_14,...,pkt_8_tcp_wsize_12,pkt_6_ipv4_cksum_4,pkt_5_ipv4_cksum_6,pkt_9_tcp_dprt_10,pkt_1_udp_sport_13,pkt_8_tcp_seq_16,pkt_2_ipv4_id_13,pkt_1_udp_cksum_6,pkt_0_ipv4_tl_12,pkt_6_udp_cksum_9
0,1,1,0,0,0,0,1,0,0,1,...,1,1,1,1,1,1,1,1,0,-1
1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,...,-1,1,1,-1,0,-1,0,0,0,1
2,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,0,0,-1,0,-1,1,1,0,0
3,1,-1,1,0,1,1,-1,1,1,1,...,1,0,1,-1,1,0,0,1,1,-1
4,0,0,0,1,1,1,1,1,0,0,...,0,0,1,1,-1,1,0,-1,0,-1
5,1,0,0,0,0,0,1,0,0,1,...,1,1,0,1,1,1,0,1,0,-1
6,0,0,0,1,1,1,0,0,1,1,...,0,0,0,1,-1,1,1,-1,0,-1
7,1,-1,1,-1,0,0,1,1,-1,1,...,0,1,1,1,-1,0,0,-1,0,1
8,0,-1,0,0,-1,-1,-1,1,0,1,...,0,0,0,-1,-1,1,0,-1,1,-1
9,0,0,0,0,0,0,0,1,0,0,...,1,1,0,1,-1,0,0,-1,0,-1


In [14]:
# Model Parameters
number_of_attributes = df_train.shape[1]
hidden_layer_nodes = 32
output_layer_nodes = 3

# Traing Parameters
EPOCHS = 250
BATCH_SIZE = 250

# Run the NN Training multiple times to get a nice mean score of accuracy results
NUMBER_OF_NN_REPETITIONS = 10

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [15]:
# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Set the 
normalizer = tf.keras.layers.Normalization(axis=-1)
# Normalize the data
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

In [16]:
# Execute Trainning and Testing
#model = get_basic_model()
train_accuracy = []
test_accuracy = []

for i in range(NUMBER_OF_NN_REPETITIONS):
  model = get_basic_model()
  history = model.fit(df_train, label_train, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
  one_train_accuracy = history.history['accuracy']

  test_loss, one_test_accuracy =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
  train_accuracy.append(one_train_accuracy[-1])
  test_accuracy.append(one_test_accuracy)
  print(f"{i+1}: train_accuracy:{train_accuracy} test_accuracy:{test_accuracy}")
  tf_predictions_probabilities = model.predict(df_test)
  #tf_loss, tf_acc =  model.evaluate(numeric_features_test,  label_test, verbose=2, batch_size=BATCH_SIZE)

  #Confusion Matrix
  tf_predictions = []
  for i,x in enumerate(tf_predictions_probabilities):
    #print("i:",i,"x_max:",x.max(),"x:",x)
    j_max = x.argmax()
    tf_predictions.append(j_max)
  conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
  print(conf_m)

#After n repetitions of training and testing, calculate statistics
train_accuracy_mean = sum(train_accuracy) / len(train_accuracy)
train_max = max(train_accuracy)
train_min = min(train_accuracy)
train_std = np.std(train_accuracy)
test_accuracy_mean = sum(test_accuracy) / len(test_accuracy)
test_max = max(test_accuracy)
test_min = min(test_accuracy)
test_std = np.std(test_accuracy)

print(f'accuracies_train: {train_accuracy}, accuracies_test: {test_accuracy}')
print(f'train_accuracy_mean: {train_accuracy_mean*100:.2f}, train_max: {train_max*100:.2f}, train_min: {train_min*100:.2f}, train_std: {train_std*100:.2f} \n test_accuracy_mean: {test_accuracy_mean*100:.2f}, test_max: {test_max*100:.2f}, test_min: {test_min*100:.2f}, test_std: {test_std*100:.2f}')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.7500 - loss: 0.6151
1: train_accuracy:[0.9873417615890503] test_accuracy:[0.75]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
tf.Tensor(
[[ 7  4  7]
 [ 0 24  0]
 [ 2  2 14]], shape=(3, 3), dtype=int32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.8000 - loss: 0.5575
2: train_accuracy:[0.9873417615890503, 0.9789029359817505] test_accuracy:[0.75, 0.800000011920929]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
tf.Tensor(
[[11  4  3]
 [ 0 23  1]
 [ 2  2 14]], shape=(3, 3), dtype=int32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.7333 - loss: 0.6151
3: train_accuracy:[0.9873417615890503, 0.9789029359817505, 0.9620253443717957] test_accuracy:[0.75, 0.800000011920929, 0.7333333492279053]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
tf.Tensor(
[[ 9  3  6]
 [ 0 21  3]
 [ 2  2 14]], shape=(3, 3), dtype=int32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.7333 - loss: 0.5786
4: train_accuracy:[0.9873417615890503, 0.9789029359817505, 0.96202534437

In [13]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "training_dataset_shape":  str(df_train.shape),
    "test_dataset_shape": str(df_test.shape),
    "number_of_attributes": number_of_attributes,
    "n_epoch":EPOCHS,
    "accuracy_train": train_accuracy_mean,
    "train_max": train_max,
    "train_min": train_min,
    "train_std": train_std,
    "accuracy_test": test_accuracy_mean,
    "test_max": test_max,
    "test_min": test_min,
    "test_std": test_std,
    "accuracies_train": train_accuracy,
    "accuracies_test": test_accuracy
}

# Writes into the file
os.chdir('../tf-model-testing')
print(os.getcwd())

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

24-06-2024-15-36-22
C:\NN-P4\tf-model-testing
